In [5]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.insert(1, '../../src')

import torch
import evaluate
import mlflow
import numpy as np

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

from TextProcessor import TextProcessor
from DatasetProcessor import DatasetProcessor


In [22]:
mlflow.set_tracking_uri("http://mlflow:5000")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

os.environ["MLFLOW_EXPERIMENT_NAME"] = "NER"
# os.environ["HF_MLFLOW_LOG_ARTIFACTS"] = "True"

# os.environ["MLFLOW_FLATTEN_PARAMS"] = "True"

# os.environ["MLFLOW_TRACKING_URI"] = tracking_uri

Current tracking uri: http://mlflow:5000


In [23]:
# raw_dataset = load_dataset("kosta-naumenko/medflex", split='train', download_mode='force_redownload', verification_mode='no_checks')
raw_train, raw_test = DatasetProcessor().make_dataset()
raw_train, raw_test

15035
Total train raws: 1725
Total test raws: 88


(Dataset({
     features: ['tokens', 'ner_tags'],
     num_rows: 1725
 }),
 Dataset({
     features: ['tokens', 'ner_tags'],
     num_rows: 88
 }))

In [24]:
model_name = "alexyalunin/RuBioRoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

In [25]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [26]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, 
        max_length=512, padding=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [27]:
train_dataset = raw_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_train.column_names,
)

test_dataset = raw_test.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_test.column_names,
)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [28]:
id = 4
input_ids, attention_mask, labels = list(train_dataset[id].values())
for i in range(len(input_ids)):
    if labels[i] > 0:
        if labels[i] == 1:
            print(" ")
        print(tokenizer.decode(input_ids[i]), end='')

 
 состояние тяжелое 
 Sat О 2 92% 
 пастозность стоп. 
 Sat О 2 92% 
 притупление в правой нижней доле 
 при аускультации - ослабленное дыхание, крепитация инспираторная в нижних отделах правого легкого 
 ЧДД - 22 в мин 
 Акцент II тона над Аортой 
 Артериальное давление 110/70 мм. рт. ст.

In [29]:
seqeval = evaluate.load("seqeval")
label_list = ['O', 'B', 'I']


def find_symtoms(result):
    text = result['text'].lower()
    symptoms = result['symptoms']
    for symptom in symptoms:
        text = text[:symptom[0]] + text[symptom[0]:symptom[1]].upper() + text[symptom[1]:]
    return text


def compute_metrics(p):
    global model, tokenizer, cur_texts, text_log_ids, processor
    model.eval()
    model.cpu()

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    for log_id in text_log_ids:

        if len(cur_texts[log_id]) == 0:
            result = processor.process_labeled_texts(
                raw_test[log_id]['tokens'],
                preds=torch.LongTensor([test_dataset[log_id]['labels']]),
                is_split=True
                )
            true_text = find_symtoms(result[0])
            cur_texts[log_id].append(true_text)

        result = processor.process_text(raw_test[log_id]['tokens'], model, is_split=True)
        cur_text = find_symtoms(result)
        cur_texts[log_id].append(cur_text)
        mlflow.log_text('\n'.join(cur_texts[log_id]), f'example_{log_id}.txt')

    model.train()
    model.to(torch.cuda.current_device())

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [30]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
}
label2id = {
    "O": 0,
    "B": 1,
    "I": 2,
}

In [31]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [34]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    device_map={'': torch.cuda.current_device()},
    cache_dir='.cache',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
    )

for param in model.roberta.parameters():
    param.requires_grad = False

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at alexyalunin/RuBioRoBERTa and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
text_log_ids = [0, 1, 2, 3, 4, 5]
cur_texts = [[] for _ in text_log_ids]
processor = TextProcessor()

cur_run_id = 8
num_train_epochs = 100

name = "RuBioRoBERTa-finetune-head"
run_name = f'{name}-{cur_run_id:02}'
output_dir = f'./logs/{run_name}'

training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_steps=1,
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to="mlflow",
    run_name=run_name
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
mlflow.end_run()

/home/knaumenko/.conda/envs/NER/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.508100,0.554042,0.002538,0.003527,0.002952,0.770875
2,0.619400,0.529487,0.002237,0.001764,0.001972,0.779249
3,0.532400,0.513047,0.003425,0.007055,0.004611,0.786466
4,0.610300,0.501889,0.004859,0.008818,0.006266,0.789935
5,0.493700,0.495015,0.005753,0.010582,0.007453,0.792009
6,0.514400,0.489849,0.003464,0.005291,0.004187,0.792328
7,0.505800,0.491572,0.005486,0.012346,0.007596,0.794202
8,0.474200,0.484378,0.004798,0.008818,0.006215,0.794003
9,0.468400,0.486989,0.004601,0.010582,0.006414,0.796076
10,0.496500,0.483966,0.007194,0.010582,0.008565,0.793086


In [36]:
path_to_model = f'../../models/{run_name}.pt'
torch.save(model, path_to_model)